In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:80% !important;}
</style>
"""))

<font size = 6>ch14 웹데이터 수집<font>

# beautifulSoup & Parser
- pip install beautifulsoup4 > anaconda에 포함
- pip install request_file

In [2]:
import requests
from requests_file import FileAdapter

In [3]:
s = requests.Session()
s.mount("file://", FileAdapter())
response = s.get("file:///BIGDATA/source/8_python/ch14_sample.html")
response

<Response [200]>

In [4]:
#html parsing
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")
soup

<html>
<head>
<title>Sample</title>
<style>
			span{color:red;}
		</style>
</head>
<body>
<h1 class="greeting css">Hello CSS</h1>
<h1 class="css">Hi CSS</h1>
<div id="subject"><b>선택자</b></div>
<p>World</p>
<div class="contents"></div>
</body>
</html> 

In [5]:
el = soup.select_one("h1") #첫번째 h1
print(el)
print(el.text)
print(el.attrs) #attrs
print(el.attrs['class']) #class속성
print(el.name)

<h1 class="greeting css">Hello CSS</h1>
Hello CSS
{'class': ['greeting', 'css']}
['greeting', 'css']
h1


In [6]:
el = soup.select("h1") #리스트로
print([e.text for e in el])
print([e.attrs for e in el])
print([e.attrs['class'] for e in el])

['Hello CSS', 'Hi CSS']
[{'class': ['greeting', 'css']}, {'class': ['css']}]
[['greeting', 'css'], ['css']]


In [7]:
soup.select("h1, p")

[<h1 class="greeting css">Hello CSS</h1>,
 <h1 class="css">Hi CSS</h1>,
 <p>World</p>]

In [8]:
soup.select("div b") #후손 선택자
soup.select("div > b") #자손 선택자: 바로 밑에 있는 b

[<b>선택자</b>]

In [9]:
soup.select("h1.greeting.css")[0]

<h1 class="greeting css">Hello CSS</h1>

In [10]:
soup.select("h1.css")

[<h1 class="greeting css">Hello CSS</h1>, <h1 class="css">Hi CSS</h1>]

# 정적 웹 데이터 수집(정적 웹크롤링)

## requests module 이용 (get 요청)

In [11]:
import requests
response = requests.get("http://api.github.com")
response

<Response [200]>

In [12]:
response.json()

{'current_user_url': 'https://api.github.com/user',
 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}',
 'authorizations_url': 'https://api.github.com/authorizations',
 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}',
 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}',
 'emails_url': 'https://api.github.com/user/emails',
 'emojis_url': 'https://api.github.com/emojis',
 'events_url': 'https://api.github.com/events',
 'feeds_url': 'https://api.github.com/feeds',
 'followers_url': 'https://api.github.com/user/followers',
 'following_url': 'https://api.github.com/user/following{/target}',
 'gists_url': 'https://api.github.com/gists{/gist_id}',
 'hub_url': 'https://api.github.com/hub',
 'issue_search_url': 'https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}',
 'issues_url': 'https://api.github.com/issues',
 'keys_url': '

## BeautifulSoup module 이용
-html이거나 xml > markup 언어를 크롤링하기 위함

### 환율 정보 가져오기

In [13]:
import requests
from bs4 import BeautifulSoup
url = "https://finance.naver.com/marketindex"
marketindex = requests.get(url)
soup = BeautifulSoup(marketindex.content, "html.parser")
price = soup.select_one("div.head_info > span.value")
print(price.text)

1,131.30


In [14]:
price = soup.select("div.head_info > span.value")
title = soup.select("h3.h_lst > span.blind")
unit = soup.select("div.head_info > span > span.blind")
unit = [u.text for u in unit]
unit.insert(7,' ')
updown = soup.select("div.head_info > span.blind")
for i in range(len(price)):
    print("{}: {}{} {}".format(title[i].text, price[i].text, unit[i], updown[i].text))

미국 USD: 1,131.30원 상승
일본 JPY(100엔): 1,037.03원 상승
유럽연합 EUR: 1,346.59원 하락
중국 CNY: 173.97원 보합
일본 엔/달러: 108.8900엔 하락
달러/유로: 1.1908달러 하락
달러/영국파운드: 1.3889달러 하락
달러인덱스: 91.8800  상승
WTI: 64.8달러 하락
휘발유: 1518.54원 상승
국제 금: 1730.6달러 상승
국내 금: 63050.74원 하락


### 네이버 영화 랭킹 출력하기

In [63]:
url = "https://movie.naver.com/movie/sdb/rank/rmovie.nhn"
movie_ranking = requests.get(url)
soup = BeautifulSoup(movie_ranking.content, "html.parser")
title_list = soup.select("td.title div.tit3 > a")
for i in range(len(title_list)):
    print("{:2d}위 {}".format(i+1, title_list[i].attrs['title']))
    link = url+title_list[i].attrs['href']
    s = BeautifulSoup(requests.get(link).content, "html.parser")
    people = s.select("li a.tx_people")
    staffs = s.select("dl.staff > dt")
    print("감독 및 배우")
    for idx in range(len(people)):
        print("{}: {}".format(staffs[idx].text, people[idx].text), end=" | ")
    print("\n")

 1위 미나리
감독 및 배우
감독: 정이삭 | 주연: 스티븐 연 | 주연: 한예리 | 주연: 윤여정 | 주연: 앨런 김 | 주연: 노엘 조 | 

 2위 극장판 귀멸의 칼날: 무한열차편
감독 및 배우
감독: 소토자키 하루오 | 주연: 하나에 나츠키 | 주연: 키토 아카리 | 조연: 시모노 히로 | 조연: 마츠오카 요시츠구 | 조연: 히노 사토시 | 

 3위 라야와 마지막 드래곤
감독 및 배우
감독: 돈 홀 | 주연: 켈리 마리 트란 | 주연: 아콰피나 | 주연: 산드라 오 | 주연: 대니얼 대 킴 | 주연: 젬마 찬 | 

 4위 미션 파서블
감독 및 배우
감독: 김형주 | 주연: 김영광 | 주연: 이선빈 | 조연: 오대환 | 조연: 서현철 | 조연: 최병모 | 

 5위 리스타트
감독 및 배우
감독: 조 카나한 | 주연: 프랭크 그릴로 | 주연: 멜 깁슨 | 주연: 나오미 왓츠 | 주연: 양자경 | 주연: 켄 정 | 

 6위 반지의 제왕: 반지 원정대
감독 및 배우
감독: 피터 잭슨 | 주연: 일라이저 우드 | 주연: 이안 맥켈런 | 주연: 리브 타일러 | 주연: 비고 모텐슨 | 주연: 숀 애스틴 | 

 7위 소울
감독 및 배우
감독: 피트 닥터 | 주연: 제이미 폭스 | 주연: 티나 페이 | 주연: 다비드 딕스 | 조연: 필리샤 라샤드 | 조연: 아미어-칼리브 톰슨 | 

 8위 중경삼림
감독 및 배우
감독: 왕가위 | 주연: 임청하 | 주연: 양조위 | 주연: 왕페이 | 주연: 금성무 | 조연: 주가령 | 

 9위 카오스 워킹
감독 및 배우
감독: 더그 라이만 | 주연: 톰 홀랜드 | 주연: 데이지 리들리 | 주연: 매즈 미켈슨 | 조연: 데이빗 오예로워 | 조연: 데미안 비쉬어 | 

10위 아이 씨 유
감독 및 배우
감독: 아담 랜달 | 주연: 헬렌 헌트 | 주연: 존 테니 | 주연: 주다 루이스 | 주연: 오웬 티그 | 주연: 리베 바러 | 

11위 암모나이트
감독 및 배우
감독: 프란시스 리 | 주연: 시얼샤 로넌 | 주연: 케이트 윈슬렛

### 로또 번호

In [61]:
url = "https://dhlottery.co.kr/gameResult.do?method=byWin"
html = requests.get(url)
soup = BeautifulSoup(html.content, "html.parser")
count = soup.select_one("div.win_result strong")
num = soup.select("div.win p span")
bonus = soup.select_one("div.bonus p span")
print(count.text,"\n당첨번호")
for i in range(len(num)):
    print("{}".format(num[i].text), end='\t')
print("+ 보너스 번호:",bonus.text)

954회 
당첨번호
1	9	26	28	30	41	+ 보너스 번호: 32


### 크롤링을 막은 사이트

In [73]:
url = "https://www.melon.com/chart/index.htm"
headers = {'User-agent':'Mozilla/5.0'}
html = requests.get(url, headers=headers)
soup = BeautifulSoup(html.content, "html.parser")
title = soup.select("div.wrap_song_info > div.ellipsis.rank01 > span > a")
singer = soup.select("div.ellipsis.rank02 > span")
for i in range(len(title)):
    print("{:3d}위: {} - {}".format(i+1, title[i].text, singer[i].text))

  1위: 롤린 (Rollin') - 브레이브걸스
  2위: Celebrity - 아이유
  3위: On The Ground - 로제 (ROSÉ)
  4위: LOVE DAY (2021) (바른연애 길잡이 X 양요섭, 정은지) - 양요섭, 정은지
  5위: Dynamite - 방탄소년단
  6위: 밤하늘의 별을(2020) - 경서
  7위: Gone - 로제 (ROSÉ)
  8위: 밝게 빛나는 별이 되어 비춰줄게 - 송이한
  9위: 내 손을 잡아 - 아이유
 10위: VVS (Feat. JUSTHIS) (Prod. GroovyRoom) - 미란이 (Mirani), 먼치맨(MUNCHMAN), Khundi Panda, 머쉬베놈 (MUSHVENOM)
 11위: 잠이 오질 않네요 - 장범준
 12위: Lovesick Girls - BLACKPINK
 13위: 취기를 빌려 (취향저격 그녀 X 산들) - 산들
 14위: 이 밤을 빌려 말해요 (바른연애 길잡이 X 10CM) - 10CM
 15위: 나랑 같이 걸을래 (바른연애 길잡이 X 적재) - 적재
 16위: 에잇(Prod.&Feat. SUGA of BTS) - 아이유
 17위: Blueming - 아이유
 18위: 흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야 - 장범준
 19위: 그날에 나는 맘이 편했을까 - 이예준
 20위: 오래된 노래 - 스탠딩 에그
 21위: 힘든 건 사랑이 아니다 - 임창정
 22위: 그 한마디 (바른연애 길잡이 X 이하이) - 이하이
 23위: 고백 (바른연애 길잡이 X 허각) - 허각
 24위: 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 - AKMU (악동뮤지션)
 25위: 사실 나는 (Feat.전건호) - 경서예지
 26위: Savage Love (Laxed - Siren Beat) (BTS Remix) - Jawsh 685, Jason Derulo, 방탄소년단
 27위: Life Goes On - 방탄소년단
 28위: METEOR - 창모 (CHANGMO)
 29위: 모든 날, 모든

## 정규표현식을 이용
간략문법
- \d (숫자와 매치. [0-9] 동일) 
- \D (숫자가 아닌 것)
- \s (whitespace)
- \w (영문자나 숫자)
- \W (영문자나 숫자가 아닌 문자)
- .  (문자 하나)
    - (1번이상 반복)
    - (0번이상 반복) {2,4} (2~4회 반복) ? ({0,1}을 의미) </pre> ### 정표현식 예
    - 메일 정규표현식 \w+@\w+.\w+[.\w+]*
    - 전화번호 정규표현식 .?\d{2,3}.?\d{3,4}.?\d{4}
    - ip주소 정규표현식 [0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}
    - 주민번호 정규표현식 [0-9]{2}[0-1][0-9][0-3][0-9]-[1-4]\d{6}

In [74]:
import re
parsing_string = """
    010-9999-9999
    01-8986
    (02)764-5502
    sa@naver.com
    sd@google.com
    <html>
        <tag>abc</tag>
    </html>
"""

In [75]:
mail_result = re.findall(r'\w+@\w+.\w+[.\w+]*', parsing_string)
mail_result

['sa@naver.com', 'sd@google.com']

In [76]:
tel_result = re.findall(r'.?\d{2,3}.?\d{3,4}.?\d{4}', parsing_string)
tel_result

[' 010-9999-9999', '(02)764-5502']

In [78]:
tag_text = re.search(r'<tag>(.+)</tag>',parsing_string)
tag_text.group(1)

'abc'

In [79]:
#일기예보 가져오기

In [96]:
import re, urllib.request
url= "http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108"
target = urllib.request.urlopen(url)
contents = target.read().decode('utf-8')
location = re.findall(r'<location wl_ver="3">.+?</location>',contents,re.DOTALL)
for loc in location:
    province = re.search(r'<province>(.+)</province>', loc)
    city = re.search(r'<city>(.+)</city>', loc)
    print(province.group(1),'지역 ',city.group(1))
    data = re.findall(r'<data>.+?</data>',loc,re.DOTALL)
    for item in data:
        tmEf = re.search(r'<tmEf>(.+)</tmEf>', item).group(1)
        wf = re.search(r'<wf>(.+)</wf>', item).group(1)
        tmn = re.search(r'<tmn>(.+)</tmn>', item).group(1)
        tmx = re.search(r'<tmx>(.+)</tmx>', item).group(1)
        print("\t날짜시간:{}, 날씨:{}, 최저:{}도, 최고:{}도".format(tmEf, wf, tmn, tmx))

서울ㆍ인천ㆍ경기도 지역  서울
	날짜시간:2021-03-20 00:00, 날씨:흐림, 최저:8도, 최고:15도
	날짜시간:2021-03-20 12:00, 날씨:흐리고 비, 최저:8도, 최고:15도
	날짜시간:2021-03-21 00:00, 날씨:구름많음, 최저:5도, 최고:10도
	날짜시간:2021-03-21 12:00, 날씨:맑음, 최저:5도, 최고:10도
	날짜시간:2021-03-22 00:00, 날씨:맑음, 최저:2도, 최고:10도
	날짜시간:2021-03-22 12:00, 날씨:맑음, 최저:2도, 최고:10도
	날짜시간:2021-03-23 00:00, 날씨:맑음, 최저:4도, 최고:14도
	날짜시간:2021-03-23 12:00, 날씨:맑음, 최저:4도, 최고:14도
	날짜시간:2021-03-24 00:00, 날씨:맑음, 최저:5도, 최고:15도
	날짜시간:2021-03-24 12:00, 날씨:맑음, 최저:5도, 최고:15도
	날짜시간:2021-03-25 00:00, 날씨:구름많음, 최저:6도, 최고:15도
	날짜시간:2021-03-26 00:00, 날씨:구름많음, 최저:6도, 최고:15도
	날짜시간:2021-03-27 00:00, 날씨:흐림, 최저:6도, 최고:15도
서울ㆍ인천ㆍ경기도 지역  인천
	날짜시간:2021-03-20 00:00, 날씨:흐림, 최저:8도, 최고:13도
	날짜시간:2021-03-20 12:00, 날씨:흐리고 비, 최저:8도, 최고:13도
	날짜시간:2021-03-21 00:00, 날씨:구름많음, 최저:5도, 최고:8도
	날짜시간:2021-03-21 12:00, 날씨:맑음, 최저:5도, 최고:8도
	날짜시간:2021-03-22 00:00, 날씨:맑음, 최저:3도, 최고:9도
	날짜시간:2021-03-22 12:00, 날씨:맑음, 최저:3도, 최고:9도
	날짜시간:2021-03-23 00:00, 날씨:맑음, 최저:5도, 최고:13도
	날짜시간:2021-03-23 12:00, 날씨:맑음, 최저:5도, 최고:13도
	날짜시간:2021-

In [108]:
#urlopen 이용
url = "http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108"
target = urllib.request.urlopen(url)
soup = BeautifulSoup(target, "html.parser")
location = soup.select("location")
for loc in location:
    province = loc.select_one("province").text
    city = loc.select_one("city").text
    print("{}의 지역 {}".format(province, city))
    data = loc.select("data")
    for idx in data:
        tmEf = idx.select_one("tmEf").text
        wf = idx.select_one("wf").text
        tmn = idx.select_one("tmn").text
        tmx = idx.select_one("tmx").text
        print("\t날짜시간:{}, 날씨:{}, 최저:{}도, 최고:{}도".format(tmEf, wf, tmn, tmx))
    print("\n")

서울ㆍ인천ㆍ경기도의 지역 서울
	날짜시간:2021-03-20 00:00, 날씨:흐림, 최저:8도, 최고:15도
	날짜시간:2021-03-20 12:00, 날씨:흐리고 비, 최저:8도, 최고:15도
	날짜시간:2021-03-21 00:00, 날씨:구름많음, 최저:5도, 최고:10도
	날짜시간:2021-03-21 12:00, 날씨:맑음, 최저:5도, 최고:10도
	날짜시간:2021-03-22 00:00, 날씨:맑음, 최저:2도, 최고:10도
	날짜시간:2021-03-22 12:00, 날씨:맑음, 최저:2도, 최고:10도
	날짜시간:2021-03-23 00:00, 날씨:맑음, 최저:4도, 최고:14도
	날짜시간:2021-03-23 12:00, 날씨:맑음, 최저:4도, 최고:14도
	날짜시간:2021-03-24 00:00, 날씨:맑음, 최저:5도, 최고:15도
	날짜시간:2021-03-24 12:00, 날씨:맑음, 최저:5도, 최고:15도
	날짜시간:2021-03-25 00:00, 날씨:구름많음, 최저:6도, 최고:15도
	날짜시간:2021-03-26 00:00, 날씨:구름많음, 최저:6도, 최고:15도
	날짜시간:2021-03-27 00:00, 날씨:흐림, 최저:6도, 최고:15도


서울ㆍ인천ㆍ경기도의 지역 인천
	날짜시간:2021-03-20 00:00, 날씨:흐림, 최저:8도, 최고:13도
	날짜시간:2021-03-20 12:00, 날씨:흐리고 비, 최저:8도, 최고:13도
	날짜시간:2021-03-21 00:00, 날씨:구름많음, 최저:5도, 최고:8도
	날짜시간:2021-03-21 12:00, 날씨:맑음, 최저:5도, 최고:8도
	날짜시간:2021-03-22 00:00, 날씨:맑음, 최저:3도, 최고:9도
	날짜시간:2021-03-22 12:00, 날씨:맑음, 최저:3도, 최고:9도
	날짜시간:2021-03-23 00:00, 날씨:맑음, 최저:5도, 최고:13도
	날짜시간:2021-03-23 12:00, 날씨:맑음, 최저:5도, 최고:13도
	날짜시간:202

	날짜시간:2021-03-21 00:00, 날씨:흐리고 비, 최저:9도, 최고:14도
	날짜시간:2021-03-21 12:00, 날씨:구름많음, 최저:9도, 최고:14도
	날짜시간:2021-03-22 00:00, 날씨:맑음, 최저:4도, 최고:13도
	날짜시간:2021-03-22 12:00, 날씨:맑음, 최저:4도, 최고:13도
	날짜시간:2021-03-23 00:00, 날씨:맑음, 최저:6도, 최고:15도
	날짜시간:2021-03-23 12:00, 날씨:맑음, 최저:6도, 최고:15도
	날짜시간:2021-03-24 00:00, 날씨:맑음, 최저:8도, 최고:15도
	날짜시간:2021-03-24 12:00, 날씨:맑음, 최저:8도, 최고:15도
	날짜시간:2021-03-25 00:00, 날씨:맑음, 최저:8도, 최고:16도
	날짜시간:2021-03-26 00:00, 날씨:구름많음, 최저:8도, 최고:15도
	날짜시간:2021-03-27 00:00, 날씨:흐림, 최저:8도, 최고:15도


대구ㆍ경상북도의 지역 대구
	날짜시간:2021-03-20 00:00, 날씨:흐림, 최저:7도, 최고:14도
	날짜시간:2021-03-20 12:00, 날씨:흐리고 비, 최저:7도, 최고:14도
	날짜시간:2021-03-21 00:00, 날씨:흐리고 비, 최저:9도, 최고:15도
	날짜시간:2021-03-21 12:00, 날씨:구름많음, 최저:9도, 최고:15도
	날짜시간:2021-03-22 00:00, 날씨:맑음, 최저:2도, 최고:13도
	날짜시간:2021-03-22 12:00, 날씨:맑음, 최저:2도, 최고:13도
	날짜시간:2021-03-23 00:00, 날씨:맑음, 최저:2도, 최고:18도
	날짜시간:2021-03-23 12:00, 날씨:맑음, 최저:2도, 최고:18도
	날짜시간:2021-03-24 00:00, 날씨:맑음, 최저:5도, 최고:18도
	날짜시간:2021-03-24 12:00, 날씨:맑음, 최저:5도, 최고:18도
	날짜시간:2021-03-25 00:00,

In [109]:
#requests 이용
url = "http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108"
target = requests.get(url)
soup = BeautifulSoup(target.content, "html.parser")
location = soup.select("location")
for loc in location:
    province = loc.select_one("province").text
    city = loc.select_one("city").text
    print("{}의 지역 {}".format(province, city))
    data = loc.select("data")
    for idx in data:
        tmEf = idx.select_one("tmEf").text
        wf = idx.select_one("wf").text
        tmn = idx.select_one("tmn").text
        tmx = idx.select_one("tmx").text
        print("\t날짜시간:{}, 날씨:{}, 최저:{}도, 최고:{}도".format(tmEf, wf, tmn, tmx))
    print("\n")

서울ㆍ인천ㆍ경기도의 지역 서울
	날짜시간:2021-03-20 00:00, 날씨:흐림, 최저:8도, 최고:15도
	날짜시간:2021-03-20 12:00, 날씨:흐리고 비, 최저:8도, 최고:15도
	날짜시간:2021-03-21 00:00, 날씨:구름많음, 최저:5도, 최고:10도
	날짜시간:2021-03-21 12:00, 날씨:맑음, 최저:5도, 최고:10도
	날짜시간:2021-03-22 00:00, 날씨:맑음, 최저:2도, 최고:10도
	날짜시간:2021-03-22 12:00, 날씨:맑음, 최저:2도, 최고:10도
	날짜시간:2021-03-23 00:00, 날씨:맑음, 최저:4도, 최고:14도
	날짜시간:2021-03-23 12:00, 날씨:맑음, 최저:4도, 최고:14도
	날짜시간:2021-03-24 00:00, 날씨:맑음, 최저:5도, 최고:15도
	날짜시간:2021-03-24 12:00, 날씨:맑음, 최저:5도, 최고:15도
	날짜시간:2021-03-25 00:00, 날씨:구름많음, 최저:6도, 최고:15도
	날짜시간:2021-03-26 00:00, 날씨:구름많음, 최저:6도, 최고:15도
	날짜시간:2021-03-27 00:00, 날씨:흐림, 최저:6도, 최고:15도


서울ㆍ인천ㆍ경기도의 지역 인천
	날짜시간:2021-03-20 00:00, 날씨:흐림, 최저:8도, 최고:13도
	날짜시간:2021-03-20 12:00, 날씨:흐리고 비, 최저:8도, 최고:13도
	날짜시간:2021-03-21 00:00, 날씨:구름많음, 최저:5도, 최고:8도
	날짜시간:2021-03-21 12:00, 날씨:맑음, 최저:5도, 최고:8도
	날짜시간:2021-03-22 00:00, 날씨:맑음, 최저:3도, 최고:9도
	날짜시간:2021-03-22 12:00, 날씨:맑음, 최저:3도, 최고:9도
	날짜시간:2021-03-23 00:00, 날씨:맑음, 최저:5도, 최고:13도
	날짜시간:2021-03-23 12:00, 날씨:맑음, 최저:5도, 최고:13도
	날짜시간:202

	날짜시간:2021-03-26 00:00, 날씨:구름많음, 최저:3도, 최고:16도
	날짜시간:2021-03-27 00:00, 날씨:흐림, 최저:3도, 최고:17도


대구ㆍ경상북도의 지역 포항
	날짜시간:2021-03-20 00:00, 날씨:흐림, 최저:9도, 최고:14도
	날짜시간:2021-03-20 12:00, 날씨:흐리고 비, 최저:9도, 최고:14도
	날짜시간:2021-03-21 00:00, 날씨:흐리고 비, 최저:9도, 최고:16도
	날짜시간:2021-03-21 12:00, 날씨:구름많음, 최저:9도, 최고:16도
	날짜시간:2021-03-22 00:00, 날씨:맑음, 최저:3도, 최고:13도
	날짜시간:2021-03-22 12:00, 날씨:맑음, 최저:3도, 최고:13도
	날짜시간:2021-03-23 00:00, 날씨:맑음, 최저:4도, 최고:18도
	날짜시간:2021-03-23 12:00, 날씨:맑음, 최저:4도, 최고:18도
	날짜시간:2021-03-24 00:00, 날씨:맑음, 최저:7도, 최고:18도
	날짜시간:2021-03-24 12:00, 날씨:맑음, 최저:7도, 최고:18도
	날짜시간:2021-03-25 00:00, 날씨:맑음, 최저:8도, 최고:17도
	날짜시간:2021-03-26 00:00, 날씨:구름많음, 최저:7도, 최고:17도
	날짜시간:2021-03-27 00:00, 날씨:흐림, 최저:7도, 최고:16도


대구ㆍ경상북도의 지역 경주
	날짜시간:2021-03-20 00:00, 날씨:흐림, 최저:6도, 최고:13도
	날짜시간:2021-03-20 12:00, 날씨:흐리고 비, 최저:6도, 최고:13도
	날짜시간:2021-03-21 00:00, 날씨:흐리고 비, 최저:8도, 최고:15도
	날짜시간:2021-03-21 12:00, 날씨:구름많음, 최저:8도, 최고:15도
	날짜시간:2021-03-22 00:00, 날씨:맑음, 최저:1도, 최고:12도
	날짜시간:2021-03-22 12:00, 날씨:맑음, 최저:1도, 최고:12도
	